In [10]:
import pandas as pd

df = pd.read_csv('data/education/사교육참여율.csv')
df.head(), df.shape, df['지역별'].unique()


(  지역별    시점  사교육참여율
 0  서울  2009    79.6
 1  서울  2010    77.5
 2  서울  2011    77.0
 3  서울  2012    73.5
 4  서울  2013    75.0,
 (250, 3),
 array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
        '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object))

In [11]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'사교육참여율': 'mean'}).round(3)
    
    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # '시점'이 datetime인지 확인 후, 형식 변경
    if pd.api.types.is_datetime64_any_dtype(monthly_df['시점']):
        monthly_df['시점'] = monthly_df['시점'].dt.strftime('%Y.%m')
    
    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
final_monthly_df = pd.concat(monthly_dfs, ignore_index=True)
final_monthly_df['사교육참여율'] = final_monthly_df['사교육참여율'].map(lambda x: f"{x:.3f}")

# 결과 확인
print(final_monthly_df.head(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# 데이터 저장
final_monthly_df.to_csv('data/education/사교육참여율(월별).csv', index=False)


         시점  사교육참여율 지역별
0   2009.01  68.100  강원
1   2009.02  67.967  강원
2   2009.03  67.833  강원
3   2009.04  67.700  강원
4   2009.05  67.567  강원
5   2009.06  67.433  강원
6   2009.07  67.300  강원
7   2009.08  67.167  강원
8   2009.09  67.033  강원
9   2009.10  66.900  강원
10  2009.11  66.767  강원
11  2009.12  66.633  강원 (2813, 3) ['강원' '경기' '경남' '경북' '광주' '대구' '대전' '부산' '서울' '세종' '울산' '인천' '전남' '전북'
 '제주' '충남' '충북']
